<a href="https://colab.research.google.com/github/NUROISEA/anime-webui-colab/blob/main/notebooks/provide_your_own_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>🌟 <code>No model WebUI Colab</code></h2>

### 🤔 How to use?

 1. Download the models and/or VAEs you want in the first cell, make sure the link you provide is the direct link to the model.
 2. Go to 🚀 `Launch web UI` and specify what models and VAEs you want to load

If things breaks, call my attention by opening an issue at this [Github Repository](https://github.com/NUROISEA/anime-webui-colab)

<h2>📚 <code>Model resources</code></h2>

<h3>Models</h3>

NoCrypt's model listing is a good list
 - https://rentry.org/ncpt_fav_models

Direct model links listing of the models I have gathered [in my repo](https://github.com/NUROISEA/anime-webui-colab)
 - https://rentry.org/anime-webui-colab-link-dump

<h3>VAEs</h3>

There is only 2 to pick from

 - Anything VAE
    - https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/Anything-V3.0.vae.pt
 - Waifu Diffusion VAE
    - https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt

In [ ]:
#@title ⏬ `Download models`

#@markdown You can run this multiple times

#@markdown I recommend downloading from HuggingFace! :)

#@markdown Direct model link
model_link = "" #@param { type: 'string' }

#@markdown Does this model have a `.yaml` file?
model_has_yaml = False #@param { type: 'boolean' }
model_yaml_link = "" #@param { type: 'string' }

#@markdown Direct VAE link
vae_link = "" #@param { type: 'string' }

models_folder = "/content/models"
vae_folder = "/content/VAE"

git_flags = "-q"
wget_flags = "-q --show-progress"

try:
  has_downloaded_at_least_once
except NameError:
  has_downloaded_at_least_once = False

if not has_downloaded_at_least_once:
  !mkdir -p {models_folder}
  !mkdir -p {vae_folder}

try:
  model_list
  vae_list
except NameError:
  model_list = [""]
  vae_list = [""]

def wgetdl(link, folder):
  !wget {wget_flags} {link} -P {folder}/ --content-disposition

if model_link != "": 
  if model_link not in model_list:
    wgetdl(model_link, models_folder)
    
    if model_has_yaml and model_yaml_link != "":
      wgetdl(model_yaml_link, models_folder)
    
    model_list.append(model_link)
  else:
    print(f"Model already downloaded.")
else:
  print("No model link entered. Downloading nothing.")

if vae_link != "":
  if vae_link not in vae_list:
    wgetdl(vae_link, vae_folder)
    
    vae_list.append(vae_link) 
  else:
    print(f"VAE already downloaded.")
else:
  print("No VAE link entered. Downloading nothing.")

print("\nList of models downloaded:")
!ls -1 {models_folder}

print("\nList of VAEs downloaded:")
!ls -1 {vae_folder}

has_downloaded_at_least_once = True

In [ ]:
#@title 🚀 `Launch web UI`

#@markdown Select a model to forefully load
model = "" #@param { type: 'string' }

#@markdown Select a VAE to forefully load
vae = "" #@param { type: 'string' }

#@markdown ----
#@markdown Use ngrok?
use_ngrok = False #@param { type:'boolean' }
#@markdown Acquire a ngrok token by following [this guide](https://github.com/NUROISEA/anime-webui-colab/wiki/Using-ngrok).
ngrok_token = "" #@param { type:'string' }
ngrok_region = "auto" #@param ["auto", "ap", "au", "eu", "in", "jp", "sa", "us"]
#@markdown If using ngrok, you will now see a `ngrok.io` link to the web UI

try:
  has_downloaded_at_least_once
except:
  sys.exit("You haven't dowloaded anything, please run the previous cell first!")

web_ui_folder = "/content/stable-diffusion-webui"
vae_folder_webui = f"{web_ui_folder}/models/VAE"
extensions_folder = f"{web_ui_folder}/extensions"
embeddings_folder = f"{web_ui_folder}/embeddings"

ui_config_dictionary = {
  "txt2img/Prompt/value": "masterpiece, best quality, ",
  "txt2img/Negative prompt/value": "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry"
}

config_dictionary = {
  "quicksettings": "sd_model_checkpoint, CLIP_stop_at_last_layers, eta_noise_seed_delta",
}

try:
  utility
except NameError:
  !wget -q https://github.com/NUROISEA/anime-webui-colab/raw/main/configs/utility.py -P /content/
  import utility

if not utility.has_run: 
  !pip install -q {utility.xformers_link}
  
  print(f'Cloning stable-diffusion-webui...')
  !git clone {git_flags} https://github.com/NUROISEA/stable-diffusion-webui

  # remove vae folder so we can symlink the vae folder we created earlier
  # no vae folder launch param :/
  !rm -rf {vae_folder_webui}
  !ln -s {vae_folder} {vae_folder_webui}

  for extension_link in utility.default_extensions:
    extension_name = extension_link.split('/')[-1]
    print(f'Cloning {extension_name}...')
    !git clone {git_flags} {extension_link} {extensions_folder}/{extension_name}
  # !git clone {git_flags} github.com/username/<repo> {extensions_folder}/<repo>

  print(f'Fetching configs...')
  for config_file in utility.default_configs:
    !wget -q {config_file} -P {web_ui_folder}/

  for embedding in utility.default_embeddings:
    !wget {wget_flags} {embedding} -P {embeddings_folder}/
  # !wget {wget_flags} site.com/embedding.pt -P {embeddings_folder}/
  
  utility.dictionary_to_json(f"{web_ui_folder}/ui-config.json", ui_config_dictionary)
  utility.dictionary_to_json(f"{web_ui_folder}/config.json", config_dictionary)
  
  utility.has_run = True

%cd {web_ui_folder}

commit_hash = utility.webui_commit_hash
!git checkout {commit_hash} &> /dev/null

arg_list = [
  "--xformers",
  f"--ckpt-dir {models_folder}"
]

if use_ngrok:
  arg_list.append(f"--ngrok {ngrok_token}")
  if ngrok_region != "auto":
    arg_list.append(f"--ngrok-region {ngrok_region}")
else:
  arg_list.append("--share")

if model != "":
  arg_list.append(f"--ckpt {models_folder}/{model}")

if vae != "":
  arg_list.append(f"--vae-path {vae_folder_webui}/{vae}")

args = " ".join(arg_list)
!python launch.py {args}

In [ ]:
#@title 💾 `Saving images`

#@markdown The zip file will be visible at the files tab.

#@markdown ----

from datetime import datetime
str_date = datetime.today().strftime('%Y-%m-%d-%H%M%S')
archive_name = f"outputs-{str_date}.zip"

%cd /content/stable-diffusion-webui
print("Zipping...")
!zip -qr /content/{archive_name} outputs
print(f"\033[92mZipped. You can now find {archive_name} at the files tab.\033[0m")

# ----

#@markdown 📲 Copy zip to Google Drive? (Recommended)
copy_to_gdrive = False #@param {type:'boolean'}
#@markdown 📂 The folder path to copy the zip file, if the above is checked, if left blank it will be in `AI/Generated`
gdrive_folder = "AI/Generated" #@param { 'type': 'string' }

if copy_to_gdrive:
  from google.colab import drive

  print("Mounting to Google Drive...")
  drive.mount('/content/drive')
  if gdrive_folder == "":
    gdrive_folder = "AI/Generated"
  
  drive_folder = f"/content/drive/MyDrive/{gdrive_folder}"
  
  !mkdir -p {drive_folder}
  !cp /content/{archive_name} {drive_folder}
  print(f"\033[92mCopied to {gdrive_folder}!\033[0m")

#@markdown ----

#@markdown 📤 Upload zip to a temporary file hosting? 
upload_to_temp = True #@param {type:'boolean'}

#@markdown 🌐 The website/host to upload to. 

#@markdown You can read more about the individual site's limits [here](https://github.com/NUROISEA/anime-webui-colab/blob/main/docs/temp_file_hosts.md). 
temp_file_host = "bashupload.com" #@param ["bashupload.com", "keep.sh", "temp.sh", "transfer.sh"]

host_dictionary = {
  "bashupload.com": "https://bashupload.com",
  "keep.sh": "https://free.keep.sh",
  "temp.sh": "https://temp.sh",
  "transfer.sh": f"https://transfer.sh/{archive_name}",
}
host_link = host_dictionary[temp_file_host] # not sure if colab can do this directly at {}

if upload_to_temp:
  print(f"Uploading to {temp_file_host}...")
  !curl -T /content/{archive_name} {host_link}  
  print("\n\033[92m^^^ This is your download link! ^^^\033[0m")

#@markdown ----

#@markdown ❌ Delete the files in the colab? (this wont delete the saved zip)
delete_local_files = False #@param {type:'boolean'}
if delete_local_files:
  %cd /content/stable-diffusion-webui
  !rm -rf outputs
  print("\033[91mDeleted the outputs folder.\033[0m")